In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train_path = "../data/mpg_train.csv"

In [3]:
train = pd.read_csv(train_path)
train.head()

,YEAR,MAKE,MODEL,VEHICLE CLASS,ENGINE SIZE,CYLINDERS,TRANSMISSION,FUEL,FUEL CONSUMPTION,HWY (L/100 km),COMB (L/100 km),COMB (mpg),EMISSIONS
0,2002,FORD,FOCUS SEDAN #,COMPACT,2.0,4,A4,X,9.1,6.8,8.1,35,186
1,2021,Porsche,Panamera Turbo S,Full-size,4.0,8,AM8,Z,15.3,11.2,13.5,21,326
2,2011,INFINITI,EX35 AWD,STATION WAGON - SMALL,3.5,6,AS7,Z,12.4,8.5,10.6,27,244
3,2005,BUICK,RENDEZVOUS AWD,SUV,3.4,6,A4,X,12.7,8.7,10.9,26,251
4,2016,ROLLS-ROYCE,GHOST EWB,FULL-SIZE,6.6,12,AS8,Z,19.2,12.4,16.1,18,373


In [4]:
def data_report(df):
    '''Esta funcion describe los campos de un dataframe de pandas de forma bastante clara, crack'''
    # Sacamos los NOMBRES
    cols = pd.DataFrame(df.columns.values, columns=["COL_N"])

    # Sacamos los TIPOS
    types = pd.DataFrame(df.dtypes.values, columns=["DATA_TYPE"])

    # Sacamos los MISSINGS
    percent_missing = round(df.isnull().sum() * 100 / len(df), 2)
    percent_missing_df = pd.DataFrame(percent_missing.values, columns=["MISSINGS (%)"])

    # Sacamos los VALORES UNICOS
    unicos = pd.DataFrame(df.nunique().values, columns=["UNIQUE_VALUES"])
    
    percent_cardin = round(unicos['UNIQUE_VALUES']*100/len(df), 2)
    percent_cardin_df = pd.DataFrame(percent_cardin.values, columns=["CARDIN (%)"])

    concatenado = pd.concat([cols, types, percent_missing_df, unicos, percent_cardin_df], axis=1, sort=False)
    concatenado.set_index('COL_N', drop=True, inplace=True)

    return concatenado.T

In [5]:
new_names = ["year", "brand", "model", "vehicle_class", "engine_size", "cylinders", "transmission", "fuel_type", "fuel_city_Lkm", "fuel_hwy_Lkm", "fuel_comb_Lkm", "fuel_comb_mpg", "co2"]

In [6]:
train.columns = new_names

In [7]:
data_report(train)

COL_N,year,brand,model,vehicle_class,engine_size,cylinders,transmission,fuel_type,fuel_city_Lkm,fuel_hwy_Lkm,fuel_comb_Lkm,fuel_comb_mpg,co2
DATA_TYPE,int64,object,object,object,float64,int64,object,object,float64,float64,float64,int64,int64
MISSINGS (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UNIQUE_VALUES,23,87,3995,32,63,9,30,5,225,151,192,59,354
CARDIN (%),0.13,0.48,22.14,0.18,0.35,0.05,0.17,0.03,1.25,0.84,1.06,0.33,1.96


#**Apuntes:**
cylinders (9m valores unicos)puede ser categórica 

In [8]:
train["cylinders"] = train["cylinders"].astype("object")

In [9]:
train.head(2)

,year,brand,model,vehicle_class,engine_size,cylinders,transmission,fuel_type,fuel_city_Lkm,fuel_hwy_Lkm,fuel_comb_Lkm,fuel_comb_mpg,co2
0,2002,FORD,FOCUS SEDAN #,COMPACT,2.0,4,A4,X,9.1,6.8,8.1,35,186
1,2021,Porsche,Panamera Turbo S,Full-size,4.0,8,AM8,Z,15.3,11.2,13.5,21,326


#**Apunte 2:** la variable model tiene demasiados datos para hacer onehotencoder. Igual podriamos eliminarla y quedarnos solo con brand?

In [10]:
train["model"].value_counts()


model
JETTA                    72
MUSTANG                  65
SILVERADO                50
ALTIMA                   49
RANGER                   49
                         ..
V70 R AWD WAGON TURBO     1
G37X AWD COUPE            1
RAV4 LIMITED/SE AWD       1
NAVIGATOR 4X4 #           1
R 320 CDI                 1
Name: count, Length: 3995, dtype: int64

In [11]:
#train = train.drop("model", axis=1)

In [12]:
#Selección categoricas 
categorical_cols = train.select_dtypes(include=['object']).columns
print(categorical_cols)

Index(['brand', 'model', 'vehicle_class', 'cylinders', 'transmission',
       'fuel_type'],
      dtype='object')


In [13]:
#Hay valores repetidos pero escritos diferente.Normalizamos:

train[categorical_cols] = train[categorical_cols].apply(lambda x: x.astype(str).str.lower())

In [14]:
data_report(train)

COL_N,year,brand,model,vehicle_class,engine_size,cylinders,transmission,fuel_type,fuel_city_Lkm,fuel_hwy_Lkm,fuel_comb_Lkm,fuel_comb_mpg,co2
DATA_TYPE,int64,object,object,object,float64,object,object,object,float64,float64,float64,int64,int64
MISSINGS (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UNIQUE_VALUES,23,52,3528,24,63,9,30,5,225,151,192,59,354
CARDIN (%),0.13,0.29,19.55,0.13,0.35,0.05,0.17,0.03,1.25,0.84,1.06,0.33,1.96


# Primer data: No reducimos columnas

In [15]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Pipeline 
cat_pipeline = Pipeline([
    ("Impute_Mode", SimpleImputer(strategy="most_frequent")),  # metemos la moda 
    ("OHEncoder", OneHotEncoder(handle_unknown="ignore"))      
])

preprocessing = ColumnTransformer(
    transformers=[
        ("cat", cat_pipeline, categorical_cols)  
    ])

train_categorical = train[categorical_cols]

train_processed = preprocessing.fit_transform(train_categorical)

#nombres de las nuevas columnas
encoded_cols = preprocessing.transformers_[0][1].named_steps['OHEncoder'].get_feature_names_out(categorical_cols)

# Reemplazamos los espacios por guiones
encoded_cols = [col.replace(" ", "_") for col in encoded_cols]

train_processed_dense = train_processed.toarray()

X_train_processed_df = pd.DataFrame(train_processed_dense, columns=encoded_cols)

X_train_processed_df.head()

,brand_acura,brand_alfa_romeo,brand_aston_martin,brand_audi,brand_bentley,brand_bmw,brand_bugatti,brand_buick,brand_cadillac,brand_chevrolet,...,transmission_av8,transmission_m4,transmission_m5,transmission_m6,transmission_m7,fuel_type_d,fuel_type_e,fuel_type_n,fuel_type_x,fuel_type_z
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [16]:
X_train_processed_df.value_counts()

brand_acura  brand_alfa_romeo  brand_aston_martin  brand_audi  brand_bentley  brand_bmw  brand_bugatti  brand_buick  brand_cadillac  brand_chevrolet  brand_chrysler  brand_daewoo  brand_dodge  brand_ferrari  brand_fiat  brand_ford  brand_genesis  brand_gmc  brand_honda  brand_hummer  brand_hyundai  brand_infiniti  brand_isuzu  brand_jaguar  brand_jeep  brand_kia  brand_lamborghini  brand_land_rover  brand_lexus  brand_lincoln  brand_maserati  brand_mazda  brand_mercedes-benz  brand_mini  brand_mitsubishi  brand_nissan  brand_oldsmobile  brand_plymouth  brand_pontiac  brand_porsche  brand_ram  brand_rolls-royce  brand_saab  brand_saturn  brand_scion  brand_smart  brand_srt  brand_subaru  brand_suzuki  brand_toyota  brand_volkswagen  brand_volvo  model_1_series_m_coupe  model_1.6el  model_1.7el  model_124_spider  model_128i_cabriolet  model_128i_coupe  model_135i_cabriolet  model_135i_coupe  model_1500  model_1500_(fuelsaver_mds)  model_1500_(mds)  model_1500_4x4  model_1500_4x4_(fuelsav

# Segundo data: Reducimos columnas agrupando por la primera palabra.

In [17]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Pipeline 
cat_pipeline = Pipeline([
    ("Impute_Mode", SimpleImputer(strategy="most_frequent")),  # metemos la moda 
    ("OHEncoder", OneHotEncoder(handle_unknown="ignore"))      
])

preprocessing = ColumnTransformer(
    transformers=[
        ("cat", cat_pipeline, categorical_cols)  
    ])

train_categorical = train[categorical_cols]

train_processed = preprocessing.fit_transform(train_categorical)

#nombres de las nuevas columnas
encoded_cols = preprocessing.transformers_[0][1].named_steps['OHEncoder'].get_feature_names_out(categorical_cols)

# Donde haya un espacio cortamos y nos quedamos con lo primero
encoded_cols = [col.split(" ")[0] for col in encoded_cols]

train_processed_dense = train_processed.toarray()

X_train_processed_df = pd.DataFrame(train_processed_dense, columns=encoded_cols)

X_train_processed_df.head()

,brand_acura,brand_alfa,brand_aston,brand_audi,brand_bentley,brand_bmw,brand_bugatti,brand_buick,brand_cadillac,brand_chevrolet,...,transmission_av8,transmission_m4,transmission_m5,transmission_m6,transmission_m7,fuel_type_d,fuel_type_e,fuel_type_n,fuel_type_x,fuel_type_z
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [18]:
X_train_processed_df

,brand_acura,brand_alfa,brand_aston,brand_audi,brand_bentley,brand_bmw,brand_bugatti,brand_buick,brand_cadillac,brand_chevrolet,...,transmission_av8,transmission_m4,transmission_m5,transmission_m6,transmission_m7,fuel_type_d,fuel_type_e,fuel_type_n,fuel_type_x,fuel_type_z
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18039,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
18040,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
18041,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
18042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


#CSV PARA VER LAS COLUMNAS 

In [ ]:
# Exportar a CSV
X_train_processed_df.to_csv('X_train_processed.csv', index=False)

# hay muchos duplicados en las columnas

In [19]:
df = X_train_processed_df.copy()

# 1. Obtener el prefijo (la parte antes del primer espacio).
#    Ej: "brand_mercedes amg 3k" => "brand_mercedes"
prefixes = df.columns.str.split(" ").str[0]

# 2. Agrupar columnas por el prefijo
grouped = df.groupby(prefixes, axis=1)

# 3. Combinar usando el máximo (OR lógico en columnas binarias)
df_combined = grouped.apply(lambda x: x.max(axis=1))

df_combined.head()


C:\Users\patri\AppData\Local\Temp\ipykernel_16276\3795793204.py:8: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped = df.groupby(prefixes, axis=1)


,brand_acura,brand_alfa,brand_aston,brand_audi,brand_bentley,brand_bmw,brand_bugatti,brand_buick,brand_cadillac,brand_chevrolet,...,vehicle_class_minivan,vehicle_class_pickup,vehicle_class_special,vehicle_class_station,vehicle_class_subcompact,vehicle_class_suv,vehicle_class_suv:,vehicle_class_two-seater,vehicle_class_van,vehicle_class_van:
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
df_combined.value_counts()

brand_acura  brand_alfa  brand_aston  brand_audi  brand_bentley  brand_bmw  brand_bugatti  brand_buick  brand_cadillac  brand_chevrolet  brand_chrysler  brand_daewoo  brand_dodge  brand_ferrari  brand_fiat  brand_ford  brand_genesis  brand_gmc  brand_honda  brand_hummer  brand_hyundai  brand_infiniti  brand_isuzu  brand_jaguar  brand_jeep  brand_kia  brand_lamborghini  brand_land  brand_lexus  brand_lincoln  brand_maserati  brand_mazda  brand_mercedes-benz  brand_mini  brand_mitsubishi  brand_nissan  brand_oldsmobile  brand_plymouth  brand_pontiac  brand_porsche  brand_ram  brand_rolls-royce  brand_saab  brand_saturn  brand_scion  brand_smart  brand_srt  brand_subaru  brand_suzuki  brand_toyota  brand_volkswagen  brand_volvo  cylinders_10  cylinders_12  cylinders_16  cylinders_2  cylinders_3  cylinders_4  cylinders_5  cylinders_6  cylinders_8  fuel_type_d  fuel_type_e  fuel_type_n  fuel_type_x  fuel_type_z  model_1  model_1.6el  model_1.7el  model_124  model_128i  model_135i  model_150

In [21]:
print(df_combined.info()) # 2º data corregido + columnas reducidas bajo la primera palabra de cada columna
print("-"*20)
print(X_train_processed_df.info()) # 1ª data corregido y sin manipular columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18044 entries, 0 to 18043
Columns: 848 entries, brand_acura to vehicle_class_van:
dtypes: float64(848)
memory usage: 116.7 MB
None
--------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18044 entries, 0 to 18043
Columns: 3648 entries, brand_acura to fuel_type_z
dtypes: float64(3648)
memory usage: 502.2 MB
None
